# Vocoder Hyperparameter Optimization

Multi-objective Bayesian optimization using Optuna's ask-and-tell interface.

**Objectives (all minimized):** SC, LogMag, Waveform L1, Mel Spec L1

**Hyperparameters:**
- Learning rate: 1e-4 to 1e-2 (log scale)
- Batch size: 32, 64, or 128
- Weight decay: 1e-6 to 1e-2 (log scale)

In [ ]:
# Run once: pip install optuna optuna-dashboard

In [23]:
import optuna

DB_PATH = "sqlite:///vocoder_study.db"
STUDY_NAME = "vocoder_v4"

study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=DB_PATH,
    directions=["minimize", "minimize", "minimize", "minimize"],  # SC, LogMag, Waveform L1, Mel L1
    load_if_exists=True
)

print(f"Study '{STUDY_NAME}' loaded/created with {len(study.trials)} existing trials")

[I 2025-11-26 14:30:02,599] A new study created in RDB with name: vocoder_v4


Study 'vocoder_v4' loaded/created with 0 existing trials


## Load Historical Trials

Edit the list below with your existing trial data, then run the cell once.

In [ ]:
historical_trials = [
    # {"params": {"lr": 1e-4, "batch_size": 32, "weight_decay": 1e-5}, "values": [SC, MagLoss, WaveL1, MelRecon]},
    {"params": {"lr": 0.0066151800383754495, "batch_size": 32, "weight_decay": 4.3049712863835176e-06, "sc_loss_weight": 0.16518143718572836, "mag_loss_weight": 3.5152047506565824, "wave_l1_loss_weight": 0.3876797963003954, "mel_recon_loss_weight": 1.5607438288858893}},
]

if historical_trials:
    for trial_data in historical_trials:
        study.enqueue_trial(trial_data["params"])
        trial = study.ask()
        if "values" in trial_data:
            study.tell(trial, trial_data["values"])
        else:
            study.tell(trial, state=optuna.trial.TrialState.FAIL)
    print(f"Loaded {len(historical_trials)} historical trials")
else:
    print("No historical trials to load (list is empty)")

No historical trials to load (list is empty)


## Get Next Suggestion

Run this cell to get parameters for your next training run.

In [ ]:
trial = study.ask()

# Define parameter spaces (only needed for generating new suggestions)
lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
sc_loss_weight = trial.suggest_float("sc_loss_weight", 0.1, 2.0)
mag_loss_weight = trial.suggest_float("mag_loss_weight", 0.1, 6.0)
wave_l1_loss_weight = trial.suggest_float("wave_l1_loss_weight", 0.1, 2.0)
mel_recon_loss_weight = trial.suggest_float("mel_recon_loss_weight", 0.1, 2.0)

print(f"--learning_rate {lr} --weight_decay {weight_decay} --sc_loss_weight {sc_loss_weight} --mag_loss_weight {mag_loss_weight} --wave_l1_loss_weight {wave_l1_loss_weight} --mel_recon_loss_weight {mel_recon_loss_weight}")
print('{"params": {"lr": '+str(lr)+', "batch_size": '+str(batch_size)+', "weight_decay": '+str(weight_decay)+', "sc_loss_weight": '+str(sc_loss_weight)+', "mag_loss_weight": '+str(mag_loss_weight)+', "wave_l1_loss_weight": '+str(wave_l1_loss_weight)+', "mel_recon_loss_weight": '+str(mel_recon_loss_weight)+'}')

--lr 0.0066151800383754495 --weight_decay 4.3049712863835176e-06 --sc_loss_weight 0.16518143718572836 --mag_loss_weight 3.5152047506565824 --wave_l1_loss_weight 0.3876797963003954 --mel_recon_loss_weight 1.5607438288858893
{"params": {"lr": 0.0066151800383754495, "batch_size": 32, "weight_decay": 4.3049712863835176e-06, "sc_loss_weight": 0.16518143718572836, "mag_loss_weight": 3.5152047506565824, "wave_l1_loss_weight": 0.3876797963003954, "mel_recon_loss_weight": 1.5607438288858893}


## Record Results

After training completes, fill in your 4 loss values and run this cell.

In [17]:
# Fill these in with your results
sc_loss = None       # Spectral Convergence
logmag_loss = None   # Log Magnitude
wave_l1 = None       # Waveform L1
mel_l1 = None        # Mel Spectrogram L1

# --- or if the run failed/crashed ---
run_failed = True

# Record the result
if run_failed:
    study.tell(trial, state=optuna.trial.TrialState.FAIL)
    print(f"Trial #{trial.number} marked as FAILED")
elif None in [sc_loss, logmag_loss, wave_l1, mel_l1]:
    print("ERROR: Fill in all 4 loss values (or set run_failed=True)")
else:
    study.tell(trial, [sc_loss, logmag_loss, wave_l1, mel_l1])
    print(f"Trial #{trial.number} recorded!")
    print(f"  SC: {sc_loss}, LogMag: {logmag_loss}, Wave L1: {wave_l1}, Mel L1: {mel_l1}")
    print(', "values": ['+str(sc_loss)+', '+str(logmag_loss)+', '+str(wave_l1)+', '+str(mel_l1)+']')

Trial #2 marked as FAILED


## View Results

In [ ]:
print(f"Total trials: {len(study.trials)}")
print(f"Completed: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")
print(f"Failed: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}")
print()

print("Pareto-optimal trials:")
print("="*80)
for t in study.best_trials:
    print(f"Trial #{t.number}")
    print(f"  Params: lr={t.params['lr']:.6f}, batch={t.params['batch_size']}, wd={t.params['weight_decay']:.6f}")
    print(f"  Losses: SC={t.values[0]:.4f}, LogMag={t.values[1]:.4f}, WaveL1={t.values[2]:.4f}, MelL1={t.values[3]:.4f}")
    print()

## All Trials Table

In [ ]:
import pandas as pd

rows = []
for t in study.trials:
    if t.state == optuna.trial.TrialState.COMPLETE:
        rows.append({
            "trial": t.number,
            "lr": t.params.get("lr"),
            "batch_size": t.params.get("batch_size"),
            "weight_decay": t.params.get("weight_decay"),
            "SC": t.values[0],
            "LogMag": t.values[1],
            "WaveL1": t.values[2],
            "MelL1": t.values[3],
        })

df = pd.DataFrame(rows)
df

## Launch Dashboard

Run this in a terminal (not in the notebook):
```bash
optuna-dashboard sqlite:///vocoder_study.db
```

Then open http://localhost:8080 in your browser.